# THINGS-fMRI usage notes

For a detailed description of the data and the procedures that generated it, see [the THINGS-data preprint](https://doi.org/10.1101/2022.07.22.501123).

In [1]:
from os.path import join as pjoin
import glob
import numpy as np
import pandas as pd
from nilearn.masking import unmask
from nilearn.plotting import plot_stat_map
from nilearn.image import load_img, index_img
import matplotlib.pyplot as plt


In [2]:
# Assumes you've downloaded the THINGS-fMRI data to this directory
# basedir = '/Users/olivercontier/bigfri/openneuro/THINGS-data/THINGS-fMRI/derivatives'

## Single trial responses

The single trial responses are arguably the easiest way to analyze the THINGS-fMRI data. They contains the magnitude of the fMRI response to each stimulus in each voxel with a single number. The single trial responses are provided in two formats: a) In table format, b) in volumetric format.

### Table format

Besides the fMRI response data, the table format contains metadata about each voxel (such as noise ceilings, pRF parameters, regions of interest) and about the stimulus (such as image file name, trial type, run and session). You can download this data [here](https://doi.org/10.25452/figshare.plus.20492835.v2).

In [3]:
# Assumes you downloaded the single trial responses in table format to this directory 
betas_csv_dir = pjoin(r'/mnt/dataset0/ldy/datasets/THINGS_fMRI/THINGS_fMRI_Single_Trial_table', 'betas_csv')

# and that you're interested in the data for the first subject
sub = '03'

In [4]:
sub

'03'

The `sub-{subject}_ResponseData.h5` files contain the actual single trial responses. Rows are voxels, columns are trials.

In [5]:
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)  # this may take a minute
print('Single trial response data')


/tmp/ipykernel_3995974/146607979.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  responses = pd.read_hdf(data_file)  # this may take a minute


Single trial response data


In [6]:
print(responses.head())

   voxel_id         0         1         2         3         4         5  \
0         0  0.013239  0.002022 -0.105115 -0.028644  0.010820  0.045174   
1         1 -0.125393 -0.003670  0.091215 -0.040909  0.069582  0.069192   
2         2  0.024484  0.065484 -0.042041 -0.014450 -0.002779  0.079440   
3         3 -0.085995  0.081165  0.064865 -0.023732  0.000979  0.053820   
4         4  0.045378  0.030528  0.052574  0.077042 -0.031905  0.029656   

          6         7         8  ...      9830      9831      9832      9833  \
0  0.021671  0.067446 -0.012943  ... -0.034353 -0.120423  0.064878 -0.089535   
1 -0.002254  0.052782  0.002731  ...  0.004860 -0.006326 -0.012317 -0.081720   
2 -0.003625  0.052408 -0.003191  ...  0.002041 -0.013647 -0.021190  0.000605   
3 -0.040727 -0.012333  0.045912  ...  0.041776  0.115980 -0.023257 -0.011929   
4 -0.072517 -0.015580  0.032488  ...  0.045142  0.064435  0.123784  0.001207   

       9834      9835      9836      9837      9838      9839  
0  0

In [7]:
print(responses.shape)

(189164, 9841)


The `sub-{subject}_VoxelMetadata.csv` files contain additional information about each voxel, such as membership to ROIs, reliability measures, and noise ceilings.

In [8]:
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
voxdata.head()

,voxel_id,subject_id,voxel_x,voxel_y,voxel_z,nc_singletrial,nc_testset,splithalf_uncorrected,splithalf_corrected,prf-eccentricity,...,glasser-p47r,glasser-TGv,glasser-MBelt,glasser-LBelt,glasser-A4,glasser-STSva,glasser-TE1m,glasser-PI,glasser-a32pr,glasser-p24
0,0,3,0,33,34,0.000000,0.000000,-0.000847,-0.001695,10.378512,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0,33,35,1.551098,15.900257,0.080501,0.149006,11.895808,...,0,0,0,0,0,0,0,0,0,0
2,2,3,0,34,34,0.000000,0.000000,-0.119311,-0.270950,8.952419,...,0,0,0,0,0,0,0,0,0,0
3,3,3,0,34,35,0.033844,0.404616,0.004827,0.009608,10.443779,...,0,0,0,0,0,0,0,0,0,0
4,4,3,0,35,34,2.784395,25.578476,0.146826,0.256057,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [9]:
print('available voxel metadata:\n', voxdata.columns.to_list())

available voxel metadata:
 ['voxel_id', 'subject_id', 'voxel_x', 'voxel_y', 'voxel_z', 'nc_singletrial', 'nc_testset', 'splithalf_uncorrected', 'splithalf_corrected', 'prf-eccentricity', 'prf-polarangle', 'prf-rsquared', 'prf-size', 'V1', 'V2', 'V3', 'hV4', 'VO1', 'VO2', 'LO1 (prf)', 'LO2 (prf)', 'TO1', 'TO2', 'V3b', 'V3a', 'lEBA', 'rEBA', 'lFFA', 'rFFA', 'lOFA', 'rOFA', 'lPPA', 'rPPA', 'lRSC', 'rRSC', 'lTOS', 'rTOS', 'lLOC', 'rLOC', 'IT', 'glasser-V1', 'glasser-MST', 'glasser-V6', 'glasser-V2', 'glasser-V3', 'glasser-V4', 'glasser-V8', 'glasser-4', 'glasser-3b', 'glasser-FEF', 'glasser-PEF', 'glasser-55b', 'glasser-V3A', 'glasser-RSC', 'glasser-POS2', 'glasser-V7', 'glasser-IPS1', 'glasser-FFC', 'glasser-V3B', 'glasser-LO1', 'glasser-LO2', 'glasser-PIT', 'glasser-MT', 'glasser-A1', 'glasser-PSL', 'glasser-SFL', 'glasser-PCV', 'glasser-STV', 'glasser-7Pm', 'glasser-7m', 'glasser-POS1', 'glasser-23d', 'glasser-v23ab', 'glasser-d23ab', 'glasser-31pv', 'glasser-5m', 'glasser-5mv', 'glasse

The voxel indices can be used to reconstruct a volume, e.g. for visualizing results obtained from the single trial responses. Alternatively, the brain mask can be used for that purpose (see below). Membership of each voxel to the available ROIs is dummy coded, e.g. in `voxdata["V1"]` or `voxdata["rFFA"]`. The population receptive field parameters are encoded in the following columns: `prf-eccentricity`, `prf-polarangle`, `prf-size`, and `prf-rsquared`. Finally, different reliability estimates are available in the columns: `nc_testset`, `nc_singletrial`, `splithalf_uncorrected`, and `splithalf_corrected`.

In [10]:
# vox_pick = voxdata['V1']
# responses_np = responses.to_numpy()
# responses_pick = responses_np

In [11]:
# V1, V2, V3, hV4, OFA, FFA, EBA, PPA, MPA and OPA
roi_columns = ['V1', 'V2', 'V3', 'hV4', 'rOFA', 'lOFA', 'rFFA', 'lFFA', 'rEBA', 'lEBA', 'rPPA', 'lPPA']
vox_pick = voxdata[roi_columns].any(axis=1)

# sum of picked true voxels
print('sum of picked true voxels:', vox_pick.sum())
print(vox_pick)


sum of picked true voxels: 5238
0         False
1         False
2         False
3         False
4         False
          ...  
189159    False
189160    False
189161    False
189162    False
189163    False
Length: 189164, dtype: bool


In [12]:
# Stimulus metadata
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)
stimdata.shape

(9840, 7)

In [13]:
import os

training_images_dir = '/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/images_set/training_images'
test_images_dir = '/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/images_set/test_images'
train_dir_names = os.listdir(training_images_dir)
test_dir_names = os.listdir(test_images_dir)
# erase the part before the first underscore
train_dir_names = [name.split('_', 1)[1] for name in train_dir_names]
test_dir_names = [name.split('_', 1)[1] for name in test_dir_names]
print(sorted(train_dir_names), len(train_dir_names))
print(sorted(test_dir_names), len(test_dir_names))

['aardvark', 'abacus', 'accordion', 'acorn', 'air_conditioner', 'air_mattress', 'air_pump', 'airbag', 'airboat', 'airplane', 'album', 'alligator', 'almond', 'aloe', 'alpaca', 'altar', 'aluminum_foil', 'amber', 'ambulance', 'amplifier', 'anchor', 'ankle', 'anklet', 'ant', 'anteater', 'antenna', 'anvil', 'appetizer', 'apple', 'apple_tree', 'applesauce', 'apron', 'aquarium', 'arch', 'arm', 'armor', 'arrow', 'artichoke', 'arugula', 'ashtray', 'asparagus', 'avocado', 'awning', 'axe', 'baby', 'backdrop', 'backgammon', 'backpack', 'bacon', 'badge', 'badger', 'bag', 'bagel', 'bagpipe', 'baklava', 'ball', 'balloon', 'ballot_box', 'bamboo', 'banana_peel', 'banana_split', 'bandage', 'bandanna', 'banjo', 'bank', 'banner', 'barbed_wire', 'barbell', 'barcode', 'bark', 'barnacle', 'barrel', 'barrette', 'baseball', 'baseball_glove', 'basket', 'basketball_hoop', 'bassinet', 'bat1', 'bat2', 'bathmat', 'bathrobe', 'bathtub', 'baton1', 'baton2', 'baton3', 'battery', 'bazooka', 'beachball', 'bead', 'beaker

In [14]:
# extract all concepts from stimulus metadata
concepts = stimdata['concept'].unique()
print('concepts:', len(concepts))

concepts: 720


In [15]:
# find concepts that are in the training set and test set
train_concepts = [concept for concept in concepts if concept in train_dir_names]
test_concepts = [concept for concept in concepts if concept in test_dir_names]
print('train_concepts:', len(train_concepts))
print('test_concepts:', len(test_concepts))

train_concepts: 633
test_concepts: 87


In [16]:
# remove all trials that trial_type is test
# stimdata = stimdata[stimdata['trial_type'] == 'train']
# print('stimdata:', stimdata.shape)

In [17]:
# # find trial_ids for test and train concepts
# train_trial_ids = stimdata[stimdata['concept'].isin(train_concepts)]['trial_id']
# test_trial_ids = stimdata[stimdata['concept'].isin(test_concepts)]['trial_id']
# print('train_trial_ids:', len(train_trial_ids))
# print('test_trial_ids:', len(test_trial_ids))

In [18]:
# apply voxel mask to responses
responses_np = responses.to_numpy()
responses_pick = responses_np[vox_pick,:]
print('responses_pick:', responses_pick.shape)

# remove the first column of responses_pick
responses_pick = responses_pick[:,1:]
print('responses_pick:', responses_pick.shape)
print('responses_pick:', responses_pick)


responses_pick: (5238, 9841)
responses_pick: (5238, 9840)
responses_pick: [[ 0.01900595  0.0229135  -0.06697899 ...  0.08979687  0.05588575
   0.04279312]
 [ 0.01402964 -0.01851454 -0.16293375 ...  0.05386849  0.12791638
   0.00577851]
 [ 0.00799971 -0.04215239 -0.06869916 ...  0.06243646  0.00178091
  -0.01686477]
 ...
 [-0.08852731 -0.07090534 -0.0025794  ... -0.01899403 -0.02433767
  -0.0199908 ]
 [ 0.12763731  0.01390089 -0.00898259 ...  0.00069999  0.08681266
  -0.00326645]
 [-0.04713013  0.02682863  0.03472633 ... -0.09340548 -0.0028976
   0.03100214]]


In [19]:
stimdata['concept'].isin(train_concepts)

0        True
1        True
2        True
3       False
4        True
        ...  
9835     True
9836     True
9837     True
9838     True
9839     True
Name: concept, Length: 9840, dtype: bool

In [20]:
stimdata['concept']

0            jack
1           sword
2          peeler
3           onion
4         go-kart
          ...    
9835    butterfly
9836         lime
9837       seesaw
9838      dustpan
9839          pea
Name: concept, Length: 9840, dtype: object

In [21]:
stimdata['concept'].isin(test_concepts)

0       False
1       False
2       False
3        True
4       False
        ...  
9835    False
9836    False
9837    False
9838    False
9839    False
Name: concept, Length: 9840, dtype: bool

In [22]:
import numpy as np

# 假设 stimdata 是一个字典或类似结构，包含 'trial_type' 和 'stimulus'
# 并且 responses_pick 是一个二维 NumPy 数组，维度为 (features, trials)
# fill train_data and test_data with responses_pick according to the sorted trial_ids one by one
test_data = responses_pick[:, stimdata['concept'].isin(test_concepts)]
train_data = responses_pick[:, stimdata['concept'].isin(train_concepts)]

# 创建布尔掩码
# test_mask = stimdata['concept'].isin(test_concepts)
test_mask = stimdata['concept'].isin(test_concepts) & (stimdata['trial_type'] != 'test')

# train_mask = stimdata['concept'].isin(train_concepts)
train_mask = stimdata['concept'].isin(train_concepts) & (stimdata['trial_type'] != 'test')

# # 分别提取 test 和 train 数据
# test_data = responses_pick[:, test_mask]
# train_data = responses_pick[:, train_mask]

# 提取对应的 stimulus
test_stimulus = stimdata['stimulus'][test_mask]
train_stimulus = stimdata['stimulus'][train_mask]

In [23]:
len(test_concepts)

87

In [24]:
test_stimulus

3              onion_11n.jpg
5           antelope_11s.jpg
9            manatee_11s.jpg
12      sleeping_bag_11s.jpg
33            possum_11s.jpg
                ...         
9765      wheelchair_12n.jpg
9793       taillight_12s.jpg
9801           tongs_12s.jpg
9817     caterpillar_12s.jpg
9825      rhinoceros_12s.jpg
Name: stimulus, Length: 1044, dtype: object

In [25]:


# 获取排序后的索引（升序）
sorted_test_indices = np.argsort(test_stimulus)
sorted_train_indices = np.argsort(train_stimulus)

# 重排 test_data 和 train_data
sorted_test_data = test_data[:, sorted_test_indices]
sorted_train_data = train_data[:, sorted_train_indices]


# 如果需要，可以将排序后的数据重新赋值回原变量
test_data = sorted_test_data
train_data = sorted_train_data

In [26]:
# map between each concept and its corresponding trial_ids
# concept_trial_ids = {}
# for concept in concepts:
#     concept_trial_ids[concept] = stimdata[stimdata['concept'] == concept]['trial_id'].to_list()
# print('concept_trial_ids:', concept_trial_ids)

In [27]:
# # split the stimulus data into training and test sets
# train_stimdata = stimdata[stimdata['concept'].isin(train_concepts)]
# test_stimdata = stimdata[stimdata['concept'].isin(test_concepts)]
# # reaaranage stimdata in order of stimulus, alphabetically
# train_stimdata = train_stimdata.sort_values(by='stimulus')
# test_stimdata = test_stimdata.sort_values(by='stimulus')
# print('train_stimdata:', train_stimdata.shape)
# print('test_stimdata:', test_stimdata.shape)

In [28]:
# # split the responses data into training and test sets one by one according to the trial_id in the stimulus data
# # get list of trial_ids for training and test
# train_trial_ids = train_stimdata['trial_id'].to_list()
# test_trial_ids = test_stimdata['trial_id'].to_list()
# print('train_trial_ids:', len(train_trial_ids))
# print('test_trial_ids:', len(test_trial_ids))
# # erase the first column of responses
# responses_pure = responses.iloc[:, 1:]
# print('responses_pure:', responses_pure.shape)
# test_responses = responses_pure.iloc[:, test_trial_ids]
# train_responses = responses_pure.iloc[:, train_trial_ids]
# print('train_responses:', train_responses.shape)
# print('test_responses:', test_responses.shape)

In [29]:
# # apply the voxel mask
# train_responses = train_responses[vox_pick]
# test_responses = test_responses[vox_pick]
# print('train_responses:', train_responses.shape)
# print('test_responses:', test_responses.shape)


In [30]:
# train_voxels = train_responses.T.values
# test_voxels = test_responses.T.values
# test_voxels.shape
# # train_voxels.shape

In [31]:
train_data = train_data.transpose()
train_data.shape

(7596, 5238)

In [32]:
test_data = test_data.transpose()
test_data.shape

(1044, 5238)

In [33]:

train_voxels = train_data.reshape(train_data.shape[0] // 12, 12, -1)
train_voxels.shape



(633, 12, 5238)

In [34]:

# 拆分 test_responses
# 计算新形状
test_voxels = test_data.reshape(test_data.shape[0] // 12,  12, -1)
test_voxels.shape

(87, 12, 5238)

In [35]:
import os
import pickle
from os.path import join as pjoin

output_dir = pjoin(f'/mnt/dataset0/ldy/datasets/fmri_dataset/Preprocessed_rerank/sub-{sub}')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 保存 train_voxels 到 pkl 文件
with open(pjoin(output_dir, 'train_responses.pkl'), 'wb') as f:
    pickle.dump(train_voxels, f)

# 保存 test_voxels 到 pkl 文件
with open(pjoin(output_dir, 'test_responses.pkl'), 'wb') as f:
    pickle.dump(test_voxels, f)

print('train_responses and test_responses saved to pkl successfully')


train_responses and test_responses saved to pkl successfully


In [ ]:
import shutil
# copy images to the output directory
origin_root = '/mnt/dataset0/ldy/4090_Workspace/4090_THINGS/osfstorage/THINGS/Images/images'
output_dir_train = '/mnt/dataset0/ldy/datasets/fmri_dataset/images_rerank/train_images'
output_dir_test = '/mnt/dataset0/ldy/datasets/fmri_dataset/images_rerank/test_images'
if not os.path.exists(output_dir_train):
    os.makedirs(output_dir_train)
if not os.path.exists(output_dir_test):
    os.makedirs(output_dir_test)
# join origin_root with concept, stimulus to get the full path of the image
train_paths = [pjoin(origin_root, concept, stimulus) for concept, stimulus in zip(stimdata['concept'][train_mask], stimdata['stimulus'][train_mask])]
test_paths = [pjoin(origin_root, concept, stimulus) for concept, stimulus in zip(stimdata['concept'][test_mask], stimdata['stimulus'][test_mask])]
print(train_paths[:5])
print(test_paths[:5])


In [ ]:
for src_path in train_paths:
    # 获取概念名称（文件路径的倒数第二部分）
    concept = src_path.split('/')[-2]  # 获取目录名作为概念
    # 创建目标子目录
    concept_dir = os.path.join(output_dir_train, concept)
    os.makedirs(concept_dir, exist_ok=True)
    # 复制文件
    shutil.copy(src_path, concept_dir)

for src_path in test_paths:
    # 获取概念名称（文件路径的倒数第二部分）
    concept = src_path.split('/')[-2]  # 获取目录名作为概念
    # 创建目标子目录
    concept_dir = os.path.join(output_dir_test, concept)
    os.makedirs(concept_dir, exist_ok=True)
    # 复制文件
    shutil.copy(src_path, concept_dir)
    
print('images copied successfully')

In [ ]:
import os

training_images_dir = "/mnt/dataset0/ldy/datasets/fmri_dataset/images_rerank/train_images"

def count_images(directory):
    total_dirs = 0
    total_images = 0


    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            total_dirs += 1

            total_images += len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])

    return total_dirs, total_images

num_dirs, num_images = count_images(training_images_dir)

print(f"There are {num_dirs} subdirectories in total")
print(f"All subdirectories together contain {num_images} images in total")


In [ ]:
import os

test_images_dir = "/mnt/dataset0/ldy/datasets/fmri_dataset/images_rerank/test_images"

def count_images(directory):
    total_dirs = 0
    total_images = 0


    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            total_dirs += 1

            total_images += len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])

    return total_dirs, total_images

num_dirs, num_images = count_images(test_images_dir)

print(f"There are {num_dirs} subdirectories in total")
print(f"All subdirectories together contain {num_images} images in total")


In [ ]:
import pickle

path = "/mnt/dataset0/ldy/datasets/fmri_dataset/Preprocessed_rerank/sub-01/train_responses.pkl"


with open(path, 'rb') as file:
    data = pickle.load(file)



fmri_data = data
# ch_names = data['ch_names']
# times = data['times']
# meg_data.shape
fmri_data.shape

In [ ]:

import pickle

path = "/mnt/dataset0/ldy/datasets/fmri_dataset/Preprocessed_rerank/sub-01/test_responses.pkl"


with open(path, 'rb') as file:
    data = pickle.load(file)



fmri_data = data
# ch_names = data['ch_names']
# times = data['times']
fmri_data.shape


The `sub-{subject}_StimulusMetadata.csv` files contain information about the file name of the image shown in each trial, which run and session a given trial occured in, and the trial_type. 

> 🚨 **Trial types**
>
> The THINGS-fMRI experiment presented participants with three different trial types:
> - `train`: Participants passively viewed an object image.
> - `test`: Same as train, but these trials belonged to a set of 200 images which were presented in each session. It's main purpose is to allow for estimating the reliability of the single trial responses in a given voxel.
> - `catch`: Participants saw a non-object image and responded with a button press. This was included to ensure participants were engaged throughout the experiment.
>
> Note: Catch trials are excluded from the single trial responses in table format as they are likely not of interest for most applications. However, catch trials are included in the volumetric format in order to make it possible to account for them in analyses.

### Select subset of trials

We can select a subset of the response data based on the stimulus type, e.g. only the repeatedly presented `test` stimuli.

In [ ]:
test_indices = stimdata.query('trial_type == "test"').index
test_responses = responses[test_indices]
test_responses.shape

As another example, we can also select data based on which object category (or `concept`) was shown. Let's select all trials with images showing a `mango`

In [ ]:
mango_indices = stimdata.query('concept == "mango"').index
mango_responses = responses[mango_indices]
mango_responses.shape

### Volume format

The single trial responses are also provided in volume format which preserves the spatial structure of the data. The data is broken up into runs and sessions, similar to the raw data files.

In [ ]:
# the directory containing the single trial responses in volume form
betas_vol_dir = pjoin(basedir, 'betas_vol', f'sub-{sub}')
# show directory content
files = glob.glob(pjoin(betas_vol_dir, '*', '*'))
files = [f.replace(basedir, '') for f in files]
print('\n'.join(files[:10]))

The `...betas.nii.gz` files are 3D+time nifti images where the time dimension corresponds to  trials. The `...conditions.tsv` contain the file names of object images for each trial.

In [ ]:
# Load responses for example run
betas_f = pjoin(betas_vol_dir, 'ses-things01', f'sub-{sub}_ses-things01_run-01_betas.nii.gz')
betas_example = load_img(betas_f)

# and plot the volume of the 3rd trial
betas_example = index_img(betas_example, 2)
g = plot_stat_map(
    betas_example, bg_img=None, annotate=False, cmap='twilight', vmax=.4,  draw_cross=False, 
)
g.title('Example: Volumetric single trial response (trial# 3)', bgcolor='white', color='black')

In [ ]:
# load the trial conditions
# Note that the volumetric single trial responses include catch trials
conds_tsv = pjoin(betas_vol_dir, 'ses-things01', f'sub-{sub}_ses-things01_run-01_conditions.tsv')
conds = pd.read_csv(conds_tsv, sep='\t').drop(columns='Unnamed: 0')
print('Content of "...conditions.tsv"')
conds.head()

## Brain masks

The brain masks indicate wether a given voxel is located in the brain or not (`1: brain`, `0: not brain`). They can be used e.g. to create a nifti volume from the results you obtained from the single trial responses.

In [ ]:
# Say you have analyzed the single trial responses in table format have produced these results
# These results are an array of n elements, where n is the number of voxels within the brain.
results = np.random.randn(responses.shape[0])
loinds = voxdata['lLOC'].astype(bool) # pretend we found activity in LOC
results[loinds] += 5

In [ ]:
# you can easily use nilearn's masking functions to transform them 
# to an image object and save it as a nifti file

# This is the provided brain mask
bmask_dir = pjoin(basedir, 'brainmasks')
bmask_f = pjoin(bmask_dir, f'sub-{sub}_space-T1w_brainmask.nii.gz')

# use the brain mask to turn your results array into a 3D image
results_img = unmask(results, bmask_f)
# plot the image to verify
g = plot_stat_map(results_img, bg_img=None, cmap='twilight', draw_cross=False, annotate=False)
# g.add_contours(bmask_f)
g.title('Example: Reconstructing volume from array-like data', bgcolor='white', color='black')
# and save it as an nifti file
results_img.to_filename('results.nii.gz')

> 🚨 **Co-registration and volumetric space**
>
> The THINGS-fMRI data was preprocessed with [fmriprep](https://fmriprep.org/en/stable/), which includes co-registration of all functional images to a high-resolution anatomical MRI image. In other words, all functional data for a given subject (including the brain masks aind regions of interest) was transformed into a common "space", meaning that a given voxel always points to the same location in the brain - with some level of imperfection.
> Of course, it's possible to analyze the data in a different space (e.g. MNI) by downloading the raw data and preprocessing it according to your needs.

## Cortical flat maps

We provide flat maps for each subject for visualizing results on a flat representation of the cortical surface with `pycortex`. Provided that you saved your results as a nifti file in the same space that the volumetric data was in, you can use the flat maps and transformation matrices we prepared. Before you can get started, check out the [pycortex documentation](https://gallantlab.github.io/pycortex/) for an explanation on how to set up your installation.

In [ ]:
# load the results we prepared above as a volume object in pycortex
results_data = np.swapaxes(load_img('results.nii.gz').get_fdata(), 0, -1)
vol_data = cortex.Volume(results_data, 'S1', 'align_auto', cmap='twilight', vmin=-6, vmax=6)
# plot with pycortex
fig = plt.figure(figsize=(8,4))
cortex.quickshow(
    vol_data, pixelwise=True, nanmean=True, colorbar_location='left', with_rois=False, fig=fig,
)
plt.title(f'Example: Visualizing results on flat maps')
plt.show()